In [5]:
# primero instalar esto
!pip install -U spacy==2.2.4

  Using cached spacy-2.2.4-cp39-cp39-linux_x86_64.whl
  Using cached blis-0.4.1-cp39-cp39-linux_x86_64.whl
  Using cached thinc-7.4.0-cp39-cp39-linux_x86_64.whl
  Using cached srsly-1.0.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.6
    Uninstalling thinc-8.1.6:
      Successfully uninstalled thinc-8.1.6
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uni

In [3]:
!pip install rantanplan

  Using cached rantanplan-0.7.1.tar.gz (807 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rantanplan: filename=rantanplan-0.7.1-py2.py3-none-any.whl size=42519 sha256=e91cf67187b97211fba72e7ac3cff6dded6bfad9c68825bd82ea53004521292c
  Stored in directory: /home/ruizfabo/.cache/pip/wheels/45/5a/b5/1cbd3e5fb214b0e5f88ed4f73ac82c2e4f86d6032ae3635314
Successfully built rantanplan


In [6]:
import spacy_affixes

In [8]:
spacy_affixes.__version__


'0.1.4'

In [6]:
!python -m spacy download es_core_news_md==2.3.1


✘ No compatible model found for 'es_core_news_md==2.3.1' (spaCy
v2.2.4).



In [8]:
!pip install -U https://github.com/explosion/spacy-models/releases/download/es_core_news_md-2.3.1/es_core_news_md-2.3.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 602.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for es-core-news-md: filename=es_core_news_md-2.3.1-py3-none-any.whl size=47447384 sha256=c8f643f49a194636f89055d15882fa364594c893c838901e7bfcc5643d4086e6
  Stored in directory: /home/ruizfabo/.cache/pip/wheels/ac/9e/58/009b585ec547a0666c0e72e44c39f42a8660fdeccd02803652
Successfully built es-core-news-md
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
  Attempting uninstall: es-core-news-md
   

In [ ]:
#https://stackoverflow.com/questions/62999811/how-to-get-an-old-release-of-a-spacy-model
# got 2.3.1, could even try with an older model so as not get the conflict warning with spacy 2.3.9

In [15]:
!python -m spacy_affixes download es

/home/ruizfabo/anaconda3/envs/py39deezy/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [1]:
import rantanplan

In [2]:
from rantanplan.core import get_scansion

In [3]:
texto_poema_old = """Ya se doran las hojas, ya las tardes
Son lo mejor del día
Bajo su luz madura de sosiego,
Ya comienza el curso.
Aprender, enseñar, lecciones, aulas.
Siempre en octubre comenzaba el año.

¡Y cuántas veces esa luz de otoño
me recordó a Fray Luis:
«Ya el tiempo nos convida
A los estudios nobles...»!"""

In [8]:
texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

In [5]:
type(texto_poema)

str

In [9]:
resultado = get_scansion(texto_poema)

In [23]:
for verso in resultado:
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) for i, ltr in enumerate(escansion) if ltr == "+"])
    print(escansion, escansion_cifras)

+--+---+-+- 1 4 8 10
-+-+-+---+- 2 4 6 10
+--+---+-+- 1 4 8 10
--+--+--++- 3 6 9 10
+--+---+-+- 1 4 8 10
-+-+-+---+- 2 4 6 10
---+-+--++- 4 6 9 10
++-+-+---+- 1 2 4 6 10
+-+--+-+-+- 1 3 6 8 10
+-+--+++-+- 1 3 6 7 8 10
+--+---+-+- 1 4 8 10
--+--+-+-+- 3 6 8 10
--+--++--+- 3 6 7 10
-+-+-+---+- 2 4 6 10


In [26]:
texto_sin_blancos = [verso for verso in texto_poema.split("\n") if verso != ""]

In [27]:
texto_sin_blancos

['Rosas sangrantes sobre el mar desflora',
 'el sol que dice adioses en la tarde,',
 'riman las aguas su canción sonora,',
 'bajo nubes de fuego el poniente arde.',
 'Vibran las cañas al chocar del viento,',
 'formando extraña y triste sinfonía,',
 'y la palmera altiva en vaivén lento',
 'es una glauca nota de armonía.',
 'Una barca se aleja lentamente,',
 'una estela de luz, un vago canto,',
 'sombras que pasan sobre el quieto mar;',
 'Y las horas se van pausadamente,',
 'mientras vierte la luz su último encanto',
 'en un intenso, pálido llamear.']

In [29]:
for idx, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) if ltr == "+" else " " for i, ltr in enumerate(escansion) ])
    print(escansion, escansion_cifras, texto_sin_blancos[idx])

+--+---+-+- 1     4       8   10   Rosas sangrantes sobre el mar desflora
-+-+-+---+-   2   4   6       10   el sol que dice adioses en la tarde,
+--+---+-+- 1     4       8   10   riman las aguas su canción sonora,
--+--+--++-     3     6     9 10   bajo nubes de fuego el poniente arde.
+--+---+-+- 1     4       8   10   Vibran las cañas al chocar del viento,
-+-+-+---+-   2   4   6       10   formando extraña y triste sinfonía,
---+-+--++-       4   6     9 10   y la palmera altiva en vaivén lento
++-+-+---+- 1 2   4   6       10   es una glauca nota de armonía.
+-+--+-+-+- 1   3     6   8   10   Una barca se aleja lentamente,
+-+--+++-+- 1   3     6 7 8   10   una estela de luz, un vago canto,
+--+---+-+- 1     4       8   10   sombras que pasan sobre el quieto mar;
--+--+-+-+-     3     6   8   10   Y las horas se van pausadamente,
--+--++--+-     3     6 7     10   mientras vierte la luz su último encanto
-+-+-+---+-   2   4   6       10   en un intenso, pálido llamear.


In [35]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) for i, ltr in enumerate(escansion) if ltr == "+"])
    print(indice+1, escansion, escansion_cifras)

1 +--+---+-+- 1 4 8 10
2 -+-+-+---+- 2 4 6 10
3 +--+---+-+- 1 4 8 10
4 --+--+--++- 3 6 9 10
5 +--+---+-+- 1 4 8 10
6 -+-+-+---+- 2 4 6 10
7 ---+-+--++- 4 6 9 10
8 ++-+-+---+- 1 2 4 6 10
9 +-+--+-+-+- 1 3 6 8 10
10 +-+--+++-+- 1 3 6 7 8 10
11 +--+---+-+- 1 4 8 10
12 --+--+-+-+- 3 6 8 10
13 --+--++--+- 3 6 7 10
14 -+-+-+---+- 2 4 6 10


In [37]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    print(indice+1, escansion)

1 +--+---+-+-
2 -+-+-+---+-
3 +--+---+-+-
4 --+--+--++-
5 +--+---+-+-
6 -+-+-+---+-
7 ---+-+--++-
8 ++-+-+---+-
9 +-+--+-+-+-
10 +-+--+++-+-
11 +--+---+-+-
12 --+--+-+-+-
13 --+--++--+-
14 -+-+-+---+-


In [38]:
for verso in resultado:
    escansion = verso["rhythm"]["stress"]
    print(escansion)


+--+---+-+-
-+-+-+---+-
+--+---+-+-
--+--+--++-
+--+---+-+-
-+-+-+---+-
---+-+--++-
++-+-+---+-
+-+--+-+-+-
+-+--+++-+-
+--+---+-+-
--+--+-+-+-
--+--++--+-
-+-+-+---+-


In [31]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) if ltr == "+" else " " for i, ltr in enumerate(escansion) ])
    print(indice+1, escansion, escansion_cifras)

1 +--+---+-+- 1     4       8   10  
2 -+-+-+---+-   2   4   6       10  
3 +--+---+-+- 1     4       8   10  
4 --+--+--++-     3     6     9 10  
5 +--+---+-+- 1     4       8   10  
6 -+-+-+---+-   2   4   6       10  
7 ---+-+--++-       4   6     9 10  
8 ++-+-+---+- 1 2   4   6       10  
9 +-+--+-+-+- 1   3     6   8   10  
10 +-+--+++-+- 1   3     6 7 8   10  
11 +--+---+-+- 1     4       8   10  
12 --+--+-+-+-     3     6   8   10  
13 --+--++--+-     3     6 7     10  
14 -+-+-+---+-   2   4   6       10  


In [14]:
escansion

'-+-+-+---+-'

In [10]:
resultado

[{'tokens': [{'word': [{'syllable': 'Ro', 'is_stressed': True},
     {'syllable': 'sas', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'san', 'is_stressed': False},
     {'syllable': 'gran', 'is_stressed': True},
     {'syllable': 'tes', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'so', 'is_stressed': False},
     {'syllable': 'bre',
      'is_stressed': False,
      'has_synalepha': True,
      'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'el', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'mar', 'is_stressed': True, 'is_word_end': True}],
    'stress_position': -1},
   {'word': [{'syllable': 'des', 'is_stressed': False},
     {'syllable': 'flo', 'is_stressed': True},
     {'syllable': 'ra', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2}],
  'phonological_groups': [{'sylla

In [ ]:
# decir cómo sacar patrones métricos a partir de ahí

# tokens (lexical syllabification) vs. phonological_groups (metrical syllables),
# then rhythm/stress has the pattern, then rhytm/length has syllable length

In [ ]:
# decir cómo leer un directorio con archivos, sacar y escribir la información
# mirar cómo hice el ejemplo en el otro capítulo